# Imports

In [ ]:
# analysis
import pandas as pd
import numpy as np

# visuals
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

# preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, PolynomialFeatures

# feature selection
from sklearn.feature_selection import VarianceThreshold, SelectPercentile

# decomposition
from sklearn.decomposition import PCA

# ensemble
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier

# logistic regression
from sklearn.linear_model import LogisticRegression

# naive bayes
from sklearn.naive_bayes import GaussianNB

# process classifier
from sklearn.gaussian_process import GaussianProcessClassifier

# neighbors
from sklearn.neighbors import KNeighborsClassifier

# neural networks
from sklearn.neural_network import MLPClassifier

# support vector machines
from sklearn.svm import LinearSVC, SVC

# train test split
from sklearn.model_selection import train_test_split, cross_validate

%matplotlib inline

# Classifiers

In [ ]:
classifiers = {
    'AdaBoostClassifier' : AdaBoostClassifier(random_state=0),
    'ExtraTreesClassifier' : ExtraTreesClassifier(n_estimators=100, random_state=0),
    'RandomForestClassifier' : RandomForestClassifier(n_estimators=100, random_state=0),
    'LogisticRegression' : LogisticRegression(solver='lbfgs', multi_class='auto', random_state=0, max_iter=200),
    'GaussianNB' : GaussianNB(),
    'GaussianProcessClassifier' : GaussianProcessClassifier(random_state=0),
    'KNeighborsClassifier' : KNeighborsClassifier(),
    'MLPClassifier' : MLPClassifier(random_state=0, max_iter=300),
    'LinearSVC' : LinearSVC(random_state=0, max_iter=1100),
    'SVC' : SVC(gamma='scale', random_state=0)
}

# Ensembles

In [ ]:
ensembles = {
    'AdaBoostClassifier' : AdaBoostClassifier(random_state=0),
    'ExtraTreesClassifier' : ExtraTreesClassifier(n_estimators=100, random_state=0),
    'RandomForestClassifier' : RandomForestClassifier(n_estimators=100, random_state=0)
}

# Data

In [ ]:
# read training data
train = pd.read_csv("train.csv")

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.head()

# Categorical Features

In [ ]:
train.describe(include=['O'])

We want to predict the `type` field.

In [ ]:
train.color.unique()

In [ ]:
# get dummy color features
train = train.join(pd.get_dummies(train.color, prefix='color'))

In [ ]:
train.head()

# Feature Importance

In [ ]:
# select X features
X = train.drop(columns=[
    'id',
    'color',
    'type',
])

# select y for training
y = train.type

# define scaler
scaler = MinMaxScaler()
rscaler = RobustScaler()

# scale X
X_ = scaler.fit_transform(X)
X_r = rscaler.fit_transform(X)

In [ ]:
fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(X_, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=X.columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

In [ ]:
fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(X_r, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=X.columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

The most important feature from all three ensemble algorithms is `hair_length`.

# EDA: `hair_length`

In [ ]:
train.type.value_counts()

In [ ]:
train_ = pd.DataFrame(X_, columns=X.columns).join(y)
train_r = pd.DataFrame(X_r, columns=X.columns).join(y)

In [ ]:
sns.pairplot(train_, hue='type', x_vars=[
    'hair_length',
    'has_soul',
], y_vars=[
    'hair_length',
    'has_soul',
], height=4, aspect=1.5);

# Feature Interactions

In [ ]:
# define poly function for interactions
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

# create interaction and cubic terms, including bias
poly_X = poly.fit_transform(X_)

# get column names from poly
poly_columns = poly.get_feature_names(X.columns)

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

In [ ]:
# define poly function for interactions
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

# create interaction and cubic terms, including bias
poly_X = poly.fit_transform(X_r)

# get column names from poly
poly_columns = poly.get_feature_names(X.columns)

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

In [ ]:
poly_train = pd.DataFrame(poly_X, columns=poly_columns).join(y)

g = sns.pairplot(poly_train, hue='type', x_vars=[
    'hair_length^3',
    'has_soul^3',
    'hair_length',
], y_vars=[
    'hair_length^3',
    'has_soul^3',
    'hair_length',
], height=4, aspect=1)

g.map_offdiag(sns.kdeplot);

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

poly_train.loc[
    poly_train.type == 'Ghost',
    'hair_length',
].plot.kde(ax=ax, label='Ghost')

poly_train.loc[
    poly_train.type == 'Ghoul',
    'hair_length',
].plot.kde(ax=ax, label='Ghoul')

poly_train.loc[
    poly_train.type == 'Goblin',
    'hair_length',
].plot.kde(ax=ax, label='Goblin')

ax.legend();

After using the `RobustScaler` and adding adding cubic interactions it can be seen that **Ghost**s' tend to have `hair_length` $\le 0$ and **Ghoul**s' have `hair_length` $\ge 0$.

**Goblin**s appear split $50$-$50$.

In [ ]:
# flag if hair length < 0 (aka < median)
train['hair_length_lt0'] = (train.hair_length < train.hair_length.median()).astype(int)

# Feature Importance Part II

In [ ]:
# select X features
X = train.drop(columns=[
    'id',
    'color',
    'type',
])

# select y for training
y = train.type

# define scaler
scaler = MinMaxScaler()
rscaler = RobustScaler()

# scale X
X_ = scaler.fit_transform(X)
X_r = rscaler.fit_transform(X)

# define poly function for interactions
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)

# create interaction and cubic terms, including bias
poly_X = poly.fit_transform(X_r)

# get column names from poly
poly_columns = poly.get_feature_names(X.columns)

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

`has_soul` still appears important by itself even after cubic interactions.

# EDA: `has_soul`

In [ ]:
poly_train = pd.DataFrame(poly_X, columns=poly_columns).join(y)

g = sns.pairplot(poly_train, hue='type', x_vars=[
    'has_soul',
    'bone_length hair_length has_soul',
], y_vars=[
    'has_soul',
    'bone_length hair_length has_soul',
], height=4, aspect=1)

g.map_offdiag(sns.kdeplot);

Seems to be a similar situation for `has_soul`:
- **Ghost**s are $\le 0$
- **Ghoul**s are $\ge 0$
- **Goblin**s are split $50$-$50$

In [ ]:
# flag if has soul < 0 (aka < median)
train['has_soul_lt0'] = (train.has_soul < train.has_soul.median()).astype(int)

In [ ]:
# counts by combination of hair_length_lt0 and has_soul_lt0
train.groupby([
    'hair_length_lt0',
    'has_soul_lt0',
    'type',
]).id.count().unstack()

# Feature Importance: Part III

In [ ]:
# select X features
X = train.drop(columns=[
    'id',
    'color',
    'type',
])

# select y for training
y = train.type

# define scaler
scaler = MinMaxScaler()
rscaler = RobustScaler()

# scale X
X_ = scaler.fit_transform(X)
X_r = rscaler.fit_transform(X)

# define poly function for interactions
poly = PolynomialFeatures(degree=3, interaction_only=True, include_bias=False)

# create interaction and cubic terms, including bias
poly_X = poly.fit_transform(X_r)

# get column names from poly
poly_columns = poly.get_feature_names(X.columns)

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

After adding the two flags, `bone_length` has shown some promise with quadratic and cubic interactions (when limiting to interaction only).

# EDA: `bone_length`

In [ ]:
poly_train = pd.DataFrame(poly_X, columns=poly_columns).join(y)

g = sns.pairplot(poly_train, hue='type', x_vars=[
    'bone_length',
    'hair_length',
], y_vars=[
    'bone_length',
    'hair_length',
], height=4, aspect=1)

g.map_offdiag(sns.kdeplot);

Similar situation:
- **Ghost**s `bone_length` $\le 0$
- **Ghoul**s `bone_length` $\ge 0$
- **Goblin**s split $50$-$50$

In [ ]:
# flag if bone length < 0 (aka < median)
train['bone_length_lt0'] = (train.bone_length < train.bone_length.median()).astype(int)

In [ ]:
# counts by combination of hair_length_lt0, has_soul_lt0, and bone_length_lt0
train.groupby([
    'hair_length_lt0',
    'has_soul_lt0',
    'bone_length_lt0',
    'type',
]).id.count().unstack()

- **Ghoul**s tend to be above the median on `hair_length`, `has_soul`, and `bone_length`.
- **Ghost**s tend to be below the median on `hair_length`, `has_soul`, and `bone_length`.
- **Goblin**s tend to be at the extremes (all above or all below).

In [ ]:
# shouldn't sum features like this as the model accounts for them better at an individual level

# # sum hair_length_lt0, has_soul_lt0, bone_length_lt0 flags
# train['lt0_sum_hair_soul_bone'] = train.hair_length_lt0 + train.has_soul_lt0 + train.bone_length_lt0

In [ ]:
# # counts by lt0_sum_hair_soul_bone
# train.groupby([
#     'lt0_sum_hair_soul_bone',
#     'type',
# ]).id.count().unstack()

# Feature Importance Part IV

In [ ]:
# select X features
X = train.drop(columns=[
    'id',
    'color',
    'type',
])

# select y for training
y = train.type

# define scaler
scaler = MinMaxScaler()
rscaler = RobustScaler()

# scale X
X_ = scaler.fit_transform(X)
X_r = rscaler.fit_transform(X)

# define poly function for interactions
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# create interaction and cubic terms, including bias
poly_X = poly.fit_transform(X_r)

# get column names from poly
poly_columns = poly.get_feature_names(X.columns)

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

`rotting_flesh` has popped up in a few places:

- ~~`rotting_flesh^2 lt0_sum_hair_soul_bone` with cubic interaction~~
- `rotting_flesh` with quadratic interaction
- `rotting_flesh` with no interaction

# EDA: `rotting_flesh`

In [ ]:
poly_train = pd.DataFrame(poly_X, columns=poly_columns).join(y)

g = sns.pairplot(poly_train, hue='type', vars=[
    'rotting_flesh',
    'hair_length',
], height=4, aspect=1)

g.map_offdiag(sns.kdeplot);

In [ ]:
# flag if rotting flesh < 0 (aka < median)
train['rotting_flesh_lt0'] = (train.rotting_flesh < train.rotting_flesh.median()).astype(int)

In [ ]:
# counts by combination of hair_length_lt0, has_soul_lt0, bone_length_lt0, rotting_flesh_lt0
train.groupby([
    'hair_length_lt0',
    'has_soul_lt0',
    'bone_length_lt0',
    'rotting_flesh_lt0',
    'type',
]).id.count().unstack()

At this rate, there could be some kind of interaction effect between all of my `lt0` features.

I have $4$ of them, so let's use 4-way interaction terms.

In [ ]:
# select X features
X = train.drop(columns=[
    'id',
    'color',
    'type',
])

# select y for training
y = train.type

# define scaler
scaler = MinMaxScaler()
rscaler = RobustScaler()

# define vt
vt = VarianceThreshold()

# define selector
selector = SelectPercentile()

# scale X
X_ = scaler.fit_transform(X)
X_r = rscaler.fit_transform(X)

# define poly function for interactions
# include interaction only
poly = PolynomialFeatures(degree=4, interaction_only=True, include_bias=False)

# create interaction and cubic terms, including bias
poly_X = poly.fit_transform(X_r)

# remove fields with no variance
poly_X = vt.fit_transform(poly_X)

# keep top 10% of best fields
poly_X = selector.fit_transform(poly_X, y)

# get boolean support on vt fields
vt_columns = vt.get_support()

# get boolean support on selector fields
selector_columns = selector.get_support()

# get column names from poly
poly_columns = np.array(poly.get_feature_names(X.columns))[vt_columns][selector_columns]

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

# Correlation

In [ ]:
# put the poly data into a dataframe
poly_df = pd.DataFrame(
    data=poly_X,
    index=X.index,
    columns=poly_columns
)

In [ ]:
# heatmap of correlation
fig, ax = plt.subplots(figsize=(16,8))

sns.heatmap(poly_df.corr(), ax=ax);

In [ ]:
# store fields with "high" correlation (> 0.6)
corr = {}

for c in poly_df.columns:
    corr[c] = poly_df.corr()[c].where(lambda x : x.abs() > 0.6).dropna()

# Principal Component Analysis

In [ ]:
# define pca (retain 99% of explainable variance)
pca = PCA(n_components=0.99)

In [ ]:
corr_pca = {}

for k,v in corr.items():
    corr_pca[k] = pca.fit_transform(poly_df.loc[:, v.index])

In [ ]:
corr_v = set()

for k,v in corr.items():
    corr_v = corr_v.union(set([k]))
    corr_v = corr_v.union(set(v.index))

In [ ]:
train_pca = pd.DataFrame()

In [ ]:
for k,v in corr_pca.items():
    train_pca = pd.concat([
        train_pca,
        pd.DataFrame(v).add_prefix(f"{k}_")
    ], axis=1)

In [ ]:
tpc = train_pca.corr()

tpc.sort_values(tpc.columns.tolist(), inplace=True)

tpc.sort_values(tpc.index.tolist(), axis=1, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))

sns.heatmap(tpc, ax=ax)

In [ ]:
train_pca = train_pca.join(y)

In [ ]:
# select X features
X = train_pca.drop(columns=[
#     'id',
#     'color',
    'type',
])

# select y for training
y = train_pca.type

# define scaler
scaler = MinMaxScaler()
rscaler = RobustScaler()

# define vt
vt = VarianceThreshold()

# define selector
selector = SelectPercentile()

# scale X
X_ = scaler.fit_transform(X)
X_r = rscaler.fit_transform(X)

# define poly function for interactions
# include interaction only
poly = PolynomialFeatures(degree=4, interaction_only=True, include_bias=False)

# create interaction and cubic terms, including bias
# poly_X = poly.fit_transform(X_r)
poly_X = X_r

# remove fields with no variance
poly_X = vt.fit_transform(poly_X)

# keep top 10% of best fields
poly_X = selector.fit_transform(poly_X, y)

# get boolean support on vt fields
vt_columns = vt.get_support()

# get boolean support on selector fields
selector_columns = selector.get_support()

# get column names from poly
# poly_columns = np.array(poly.get_feature_names(X.columns))[vt_columns][selector_columns]
poly_columns = np.array(X.columns)[vt_columns][selector_columns]

fi = {}

for k,v in ensembles.items():
    fi[k] = v.fit(poly_X, y).feature_importances_

fi = pd.DataFrame.from_dict(fi, orient='index', columns=poly_columns)

fi.sort_values(fi.columns.tolist(), ascending=False, inplace=True)
fi.sort_values(fi.index.tolist(), axis=1, ascending=False, inplace=True)

fi.style.highlight_max(axis=1)

In [ ]:
g = sns.pairplot(train_pca, hue='type', vars=[
    'hair_length_lt0 has_soul_lt0_0',
    'hair_length has_soul hair_length_lt0 has_soul_lt0_0',
    'bone_length bone_length_lt0_0'
], height=4, aspect=1.5)

g.map_offdiag(sns.kdeplot);

# Model

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_pca.drop(columns=[
    'type'
]), train_pca.type, test_size=0.1, random_state=0)

In [ ]:
results = {}

for k,v in classifiers.items():
    
    print(k)
    
    cv = cross_validate(
        estimator=v,
        X=train_x,
        y=train_y,
        cv=10
    )
    
    cv = cv['test_score']
    
    results[k] = cv

In [ ]:
pd.DataFrame.from_dict(results, orient='columns').describe()